**Functions for Manual Encoding**

In [1]:
def OS_transform(df,col):
    df.loc[df[col].str.contains('Windows', na=False), col] = 'Windows'
    df.loc[df[col].str.contains('iOS', na=False), col] = 'iOS'
    df.loc[df[col].str.contains('Mac OS', na=False), col] = 'Mac'
    df.loc[df[col].str.contains('Android', na=False), col] = 'Android'
    df[col].fillna("Missing", inplace=True)
    return df

In [2]:
def mail_transform(df,col):
    df.loc[df[col].str.contains('gmail', na=False), col] = 'Google'
    df.loc[df[col].str.contains('yahoo', na=False), col] = 'Yahoo'
    df.loc[df[col].isin(['hotmail.com','outlook.com','msn.com', 'live.com.mx', 'hotmail.es','hotmail.co.uk', 'hotmail.de', 'outlook.es', 'live.com', 'live.fr', 'hotmail.fr']), col] = 'Microsoft'
    df.loc[df[col].str.contains('outlook', na=False), col] = 'Microsoft'
    df.loc[df[col].str.contains('msn', na=False), col] = 'Microsoft'
    df.loc[df[col].str.contains('live', na=False), col] = 'Microsoft'
    df[col].fillna("Missing", inplace=True)
    return df

In [3]:
def browser_transform(df,col):
    df.loc[df[col].str.contains('opera', na=False), col] = 'Opera'
    df.loc[df[col].str.contains('safari', na=False), col] = 'Safari'
    df.loc[df[col].str.contains('edge', na=False), col] = 'Microsoft'
    df.loc[df[col].str.contains('chrome', na=False), col] = 'Google Chrome'
    df.loc[df[col].str.contains('firefox', na=False), col] = 'Firefox'
    df[col].fillna("Missing", inplace=True)
    return df

In [4]:
def resol_transform(df,col):
    df.loc[df[col].isin(['3839x2160', '3001x2000', '4500x3000', '3000x2000', '4096x2304', '5120x2880', '3840x2160', '2732x2048', '3360x2100']), col] = '4K'
    df.loc[df[col].isin(['2048x1280', '3840x1080', '2049x1152', '1919x1199', '2000x1125', '2672x1440', '2700x1800', '2880x1440', '2561x1442', '1919x1080', '2400x1350', '2562x1442', '2076x1080', '3440x1440', '2112x1188', '2560x1080', '1921x1081', '1920x1281', '3201x1800', '2160x1440', '2960x1440', '2961x1442', '3200x1800', '2224x1668', '2736x1824', '1600x1200', '2220x1081', '2220x1080', '1920x1200', '2880x1800', '2560x1440', '2001x1125', '2560x1600', '2048x1536', '2436x1125', '2208x1242', '1920x1080']), col] = 'Full HD'
    df.loc[df[col].isin(['1638x922', '1280x1023', '1680x1051', '1440x759', '1600x899', '1728x972', '1344x756', '1279x1024', '1679x1049', '1365x767', '1600x837', '1280x1025', '1184x720', '1440x810', '1502x844', '1768x992', '1916x901', '1919x1079', '1920x975', '1280x768', '1536x864', '1152x864', '1366x767', '1364x768', '1365x768', '1360x768', '1024x768', '1680x1050', '1280x1024', '1280x800', '1280x720', '1600x900', '1440x900', '1334x750', '1366x768']), col] = 'HD'
    df.loc[df[col].isin(['1062x630', '800x600', '1280x712', '1024x640', '960x540', '1024x552', '640x360', '801x480', '1024x600', '855x480', '1136x640']),col] = 'SD'
    df[col].fillna("Missing", inplace=True)
    return df

In [5]:
def device_transform(df,col):
    df.loc[df[col].isin(df[col].value_counts()[df[col].value_counts() <= 500 ].index), col] = 'Others'
    df[col].fillna("Missing", inplace=True)
    return df

**Overview Function**

In [6]:
def overview(df):
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values 
    summary['Missing_Percent'] = (df.isnull().sum().values/len(df))*100
    summary['Uniques'] = df.nunique().values
    return summary

## <font color='green'> Importing Required Libraries

In [7]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option("display.max_rows", None, "display.max_columns", None)
from sklearn.model_selection import train_test_split, GridSearchCV
import lightgbm as lgb
from sklearn.metrics import confusion_matrix,accuracy_score, roc_curve, auc
sns.set_style("whitegrid")

In [8]:
class Font:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

### <font color = 'green'> Loading Sample Data

In [9]:
df = pd.read_csv('resampled_train.csv',index_col='TransactionID')

#### <font color= 'brown'> Splitting the Data to Train, Validation and Test Data**

In [10]:
y = df['isFraud']
X = df.drop('isFraud', axis = 1)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [12]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

In [13]:
print(Font.BLUE, Font.BOLD, "Shape of Train Data      : ",Font.END, Font.BOLD, X_train.shape)
print(Font.BLUE, Font.BOLD, "Shape of Validation Data : ", Font.END, Font.BOLD, X_val.shape)
print(Font.BLUE, Font.BOLD, "Shape of Test Data       : ", Font.END, Font.BOLD, X_test.shape)

  Shape of Train Data      :    (24707, 432)
  Shape of Validation Data :    (8236, 432)
  Shape of Test Data       :    (8236, 432)


#### <font color= 'brown'> Creating a List for Categorical Variables

* **cat_var        :** List of All Categorical Variables

In [14]:
cat_var = ['card4', 'card6', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'dist1', 'dist2', 'D2', 'D3', 'D11', 'D9', 'D12', 'D13', 'D14', 'D5', 'D6', 'D7', 'D8', 'id_01', 'id_03', 'id_04', 'id_05', 'id_06', 'id_07', 'id_08', 'id_09', 'id_10', 'id_11', 'id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'P_emaildomain', 'ProductCD', 'R_emaildomain', 'V1', 'V10', 'V100', 'V101', 'V102', 'V103', 'V104', 'V105', 'V106', 'V107', 'V108', 'V109', 'V11', 'V110', 'V111', 'V112', 'V113', 'V114', 'V115', 'V116', 'V117', 'V118', 'V119', 'V12', 'V120', 'V121', 'V122', 'V123', 'V124', 'V125', 'V126', 'V127', 'V128', 'V129', 'V13', 'V130', 'V131', 'V132', 'V133', 'V134', 'V135', 'V136', 'V137', 'V138', 'V139', 'V14', 'V140', 'V141', 'V142', 'V143', 'V144', 'V145', 'V146', 'V147', 'V148', 'V149', 'V15', 'V150', 'V151', 'V152', 'V153', 'V154', 'V155', 'V156', 'V157', 'V158', 'V159', 'V16', 'V160', 'V161', 'V162', 'V163', 'V164', 'V165', 'V166', 'V167', 'V168', 'V169', 'V17', 'V170', 'V171', 'V172', 'V173', 'V174', 'V175', 'V176', 'V177', 'V178', 'V179', 'V18', 'V180', 'V181', 'V182', 'V183', 'V184', 'V185', 'V186', 'V187', 'V188', 'V189', 'V19', 'V190', 'V191', 'V192', 'V193', 'V194', 'V195', 'V196', 'V197', 'V198', 'V199', 'V2', 'V20', 'V200', 'V201', 'V202', 'V203', 'V204', 'V205', 'V206', 'V207', 'V208', 'V209', 'V21', 'V210', 'V211', 'V212', 'V213', 'V214', 'V215', 'V216', 'V217', 'V218', 'V219', 'V22', 'V220', 'V221', 'V222', 'V223', 'V224', 'V225', 'V226', 'V227', 'V228', 'V229', 'V23', 'V230', 'V231', 'V232', 'V233', 'V234', 'V235', 'V236', 'V237', 'V238', 'V239', 'V24', 'V240', 'V241', 'V242', 'V243', 'V244', 'V245', 'V246', 'V247', 'V248', 'V249', 'V25', 'V250', 'V251', 'V252', 'V253', 'V254', 'V255', 'V256', 'V257', 'V258', 'V259', 'V26', 'V260', 'V261', 'V262', 'V263', 'V264', 'V265', 'V266', 'V267', 'V268', 'V269', 'V27', 'V270', 'V271', 'V272', 'V273', 'V274', 'V275', 'V276', 'V277', 'V278', 'V279', 'V28', 'V280', 'V281', 'V282', 'V283', 'V284', 'V285', 'V286', 'V287', 'V288', 'V289', 'V29', 'V290', 'V291', 'V292', 'V293', 'V294', 'V295', 'V296', 'V297', 'V298', 'V299', 'V3', 'V30', 'V300', 'V301', 'V302', 'V303', 'V304', 'V305', 'V306', 'V307', 'V308', 'V309', 'V31', 'V310', 'V311', 'V312', 'V313', 'V314', 'V315', 'V316', 'V317', 'V318', 'V319', 'V32', 'V320', 'V321', 'V322', 'V323', 'V324', 'V325', 'V326', 'V327', 'V328', 'V329', 'V33', 'V330', 'V331', 'V332', 'V333', 'V334', 'V335', 'V336', 'V337', 'V338', 'V339', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V4', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V5', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V6', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69', 'V7', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V79', 'V8', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V86', 'V87', 'V88', 'V89', 'V9', 'V90', 'V91', 'V92', 'V93', 'V94', 'V95', 'V96', 'V97', 'V98', 'V99', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo']

#### <font color= 'brown'> Creating a List for Numerical Variables

* **num_var        :** List of All Numerical Variables

In [15]:
num_var = ['TransactionAmt', 'card1', 'card2', 'card3', 'card5', 'addr1', 'addr2', 'dist1', 'dist2', 'id_02', 'D1', 'D2', 'D3', 'D4', 'D10', 'D11', 'D15']

#### <font color= 'brown'> Custom Value Imputaion List, based on category type & Suggested imputation

* **cv_impute_num  :** Numeric Type - Categorical List, for Custom Value Imputation
* **cv_impute_char :** Character Type - Categorical List, for Custom Value Imputation
* **num_impute     :** List of All Numerical Variables, which undergo missing value treatment. Ex: Mean, Median etc.,

In [140]:
cv_impute_num = [ 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'dist1', 'dist2', 'D2', 'D3', 'D11', 'D12', 'D13', 'D14', 'D5', 'D6', 'D7', 'D8', 'D9',  'V1', 'V10', 'V100', 'V101', 'V102', 'V103', 'V104', 'V105', 'V106', 'V107', 'V108', 'V109', 'V11', 'V110', 'V111', 'V112', 'V113', 'V114', 'V115', 'V116', 'V117', 'V118', 'V119', 'V12', 'V120', 'V121', 'V122', 'V123', 'V124', 'V125', 'V126', 'V127', 'V128', 'V129', 'V13', 'V130', 'V131', 'V132', 'V133', 'V134', 'V135', 'V136', 'V137', 'V138', 'V139', 'V14', 'V140', 'V141', 'V142', 'V143', 'V144', 'V145', 'V146', 'V147', 'V148', 'V149', 'V15', 'V150', 'V151', 'V152', 'V153', 'V154', 'V155', 'V156', 'V157', 'V158', 'V159', 'V16', 'V160', 'V161', 'V162', 'V163', 'V164', 'V165', 'V166', 'V167', 'V168', 'V169', 'V17', 'V170', 'V171', 'V172', 'V173', 'V174', 'V175', 'V176', 'V177', 'V178', 'V179', 'V18', 'V180', 'V181', 'V182', 'V183', 'V184', 'V185', 'V186', 'V187', 'V188', 'V189', 'V19', 'V190', 'V191', 'V192', 'V193', 'V194', 'V195', 'V196', 'V197', 'V198', 'V199', 'V2', 'V20', 'V200', 'V201', 'V202', 'V203', 'V204', 'V205', 'V206', 'V207', 'V208', 'V209', 'V21', 'V210', 'V211', 'V212', 'V213', 'V214', 'V215', 'V216', 'V217', 'V218', 'V219', 'V22', 'V220', 'V221', 'V222', 'V223', 'V224', 'V225', 'V226', 'V227', 'V228', 'V229', 'V23', 'V230', 'V231', 'V232', 'V233', 'V234', 'V235', 'V236', 'V237', 'V238', 'V239', 'V24', 'V240', 'V241', 'V242', 'V243', 'V244', 'V245', 'V246', 'V247', 'V248', 'V249', 'V25', 'V250', 'V251', 'V252', 'V253', 'V254', 'V255', 'V256', 'V257', 'V258', 'V259', 'V26', 'V260', 'V261', 'V262', 'V263', 'V264', 'V265', 'V266', 'V267', 'V268', 'V269', 'V27', 'V270', 'V271', 'V272', 'V273', 'V274', 'V275', 'V276', 'V277', 'V278', 'V279', 'V28', 'V280', 'V281', 'V282', 'V283', 'V284', 'V285', 'V286', 'V287', 'V288', 'V289', 'V29', 'V290', 'V291', 'V292', 'V293', 'V294', 'V295', 'V296', 'V297', 'V298', 'V299', 'V3', 'V30', 'V300', 'V301', 'V302', 'V303', 'V304', 'V305', 'V306', 'V307', 'V308', 'V309', 'V31', 'V310', 'V311', 'V312', 'V313', 'V314', 'V315', 'V316', 'V317', 'V318', 'V319', 'V32', 'V320', 'V321', 'V322', 'V323', 'V324', 'V325', 'V326', 'V327', 'V328', 'V329', 'V33', 'V330', 'V331', 'V332', 'V333', 'V334', 'V335', 'V336', 'V337', 'V338', 'V339', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V4', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V5', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V6', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69', 'V7', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V79', 'V8', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V86', 'V87', 'V88', 'V89', 'V9', 'V90', 'V91', 'V92', 'V93', 'V94', 'V95', 'V96', 'V97', 'V98', 'V99']
cv_impute_char = ['ProductCD', 'P_emaildomain', 'R_emaildomain', 'card4', 'card6',  'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'id_30', 'id_31', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo', 'id_01', 'id_03', 'id_04', 'id_05', 'id_06', 'id_07', 'id_08', 'id_09', 'id_10', 'id_11', 'id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29','id_32', 'id_33', 'id_34']
num_impute = ['TransactionAmt', 'card1', 'card2', 'card3', 'card5', 'addr1', 'addr2', 'id_02', 'D1', 'D4', 'D10', 'D15']

#### <font color= 'brown'> List for rare category Imputation

* **rare_5         :** List of variables to undergo Rare Encoding with 5%
* **rare_10        :** List of variables to undergo Rare Encoding with 10%

In [17]:
rare_5 = ['dist1', 'dist2', 'D2', 'D3', 'D11', 'C3', 'C4', 'C5', 'C7', 'C8', 'C9', 'C12', 'V252', 'V253', 'V254', 'V260', 'V261', 'V262', 'V281', 'V286', 'V287', 'V288', 'V289', 'V290', 'V297', 'V299', 'V300', 'V301', 'V302', 'V303', 'V304', 'V325', 'V326', 'V327', 'V328', 'V329', 'V330', 'V263', 'V264', 'V265', 'V266', 'V267', 'V268', 'V269', 'V270', 'V271', 'V272', 'V273', 'V274', 'V275', 'V276', 'V277', 'V278', 'V322', 'V323', 'V324', 'V331', 'V332', 'V333', 'V334', 'V335', 'V336', 'V337', 'V338', 'V339', 'V279', 'V283', 'V291', 'V292', 'V306', 'V307', 'V308', 'V309', 'V310', 'V311', 'V312', 'V313', 'V314', 'V315', 'V316', 'V317', 'V318', 'V319', 'V320', 'V321', 'card4', 'card6', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V79', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V86', 'V87', 'V88', 'V89', 'V90', 'V91', 'V92', 'V93', 'V94', 'id_01', 'id_03', 'id_04', 'id_05', 'id_06', 'id_07', 'id_08', 'id_09', 'id_10', 'id_11', 'id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'V98', 'V104', 'V106', 'V107', 'V108', 'V109', 'V110', 'V111', 'V112', 'V113', 'V114', 'V115', 'V116', 'V117', 'V118', 'V119', 'V120', 'V121', 'V122', 'V123', 'V125',  'V96', 'V97', 'V99', 'V101', 'V102', 'V103', 'V105', 'V126', 'V127', 'V128', 'V129', 'V130', 'V131', 'V132', 'V133', 'V134', 'V135', 'V136', 'V137', 'V143', 'V144', 'V145', 'V150', 'V151', 'V159', 'V160', 'V161', 'V162', 'V163', 'V164', 'V165', 'V166', 'V167', 'V168', 'V169', 'V170', 'V171', 'V172', 'V173', 'V174',  'V175', 'V176', 'V177', 'V178', 'V179', 'V180', 'V181', 'V182',  'V183', 'V184', 'V185', 'V186', 'V187', 'V188', 'V189', 'V190', 'V191', 'V192', 'V193', 'V194', 'V195', 'V196', 'V197', 'V198', 'V199', 'V200', 'V201', 'V202', 'V203', 'V204', 'V205', 'V206', 'V207', 'V208', 'V209', 'V210', 'V211', 'V212', 'V213', 'V214',  'V215', 'V216', 'V217', 'V218', 'V219', 'V220', 'V223', 'V224', 'V225', 'V226', 'V227', 'V228', 'V229', 'V230','V231','V232', 'V233', 'V234', 'V235', 'V236', 'V237', 'V238','V239', 'V240', 'V241', 'V242', 'V243', 'V244', 'V245', 'V246', 'V247', 'V248', 'V249', 'D5', 'D7', 'D8', 'D9', 'D14', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo']
rare_10 = ['C1', 'C2', 'C6', 'C10', 'C11', 'C13', 'C14', 'V255', 'V256', 'V282', 'V284', 'V257', 'V258', 'V259', 'V280', 'V285', 'V293', 'V294', 'V295', 'V296', 'V298', 'ProductCD', 'P_emaildomain', 'R_emaildomain', 'V12', 'V25',  'V124', 'V139', 'V140', 'V148', 'V149', 'V153', 'V154', 'V155', 'V156', 'V157', 'V158',  'V95', 'V221', 'V222', 'V245', 'V250', 'V251', 'D6', 'D12', 'D13', 'M2', 'M7', 'M9']

#### <font color= 'brown'>  Drop Variable List

* **drop_var        :** List of variables to be dropped

In [18]:
drop_var = ['V305', 'id_24', 'id_25']

#### <font color= 'brown'> Edge Variables

* **edge_var        :** Variables at the Edge to be analysed

In [19]:
edge_var = ['V4','V5','V39','V40','V42','V43','V50','V51','V52''V57','V58','V69','V60','V71','V74','V53','V54','V55','V56','V57','V58','V59','V60','V61','V62','V63','V64','V65','V66','V67', 'V68','V69','V70']

#### <font color= 'brown'> Missing Value Labelling

**miss_label  :** Missing label for group of variables

* **V1 :**  [V1-V11]
* **V12 :**  [V12-V34]
* **V35 :**  [V35-V52]
* **V53 :**  [V53-V74]
* **V75 :**  [V75-V94]


In [20]:
miss_label = ['V1', 'V12', 'V35', 'V53', 'V75']

In [185]:
df_list = X_train[cv_impute_num].describe(percentiles=[.1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .98])

In [188]:
df_list

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,dist1,dist2,D2,D3,D11,D12,D13,D14,D5,D6,D7,D8,D9,V1,V10,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V11,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V12,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V13,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V14,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V15,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V16,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V17,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V18,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V19,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V2,V20,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V21,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V22,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V23,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V24,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V25,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V26,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V27,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V28,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V29,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V3,V30,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V31,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V32,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V33,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,V34,V35,V36,V37,V38,V39,V4,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V5,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V6,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V7,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V8,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V9,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99
count,24707.000000,24707.000000,24707.000000,24707.000000,24707.000000,24707.000000,24707.000000,24707.000000,24707.000000,24707.000000,24707.000000,24707.000000,24707.000000,24707.000000,7825.000000,2961.000000,11027.000000,12433.000000,10233.000000,5755.000000,5232.000000,5489.000000,12160.000000,6005.000000,4123.00000,6176.000000,6176.000000,10233.000000,10233.000000,24695.000000,24695.000000,24695.000000,24695.000000,24695.000000,24695.000000,24695.000000,24695.000000,24695.000000,24695.000000,10233.000000,24695.000000,24695.000000,24695.000000,24695.000000,24695.000000,24695.000000,24695.000000,24695.000000,24695.000000,24695.000000,20742.000000,24695.000000,24695.000000,24695.000000,24695.000000,24695.000000,24695.000000,24695.000000,24695.000000,24695.000000,24695.000000,20742.000000,24695.000000,24695.000000,24695.000000,24695.000000,24695.000000,24695.000000,24695.000000,24695.000000,3956.00000,3956.000000,20742.000000,3956.000000,3956.000000,3956.000000,3957.000000,3957.000000,3957.000000,3956.000000,3956.000000,3956.000000,3956.000000,20742.000000,3957.000000,3957.000000,3957.000000,3956.000000,3956.000000,3956.000000,3956.000000,3956.000000,3956.000000,3957.000000,20742.000000,3957.000000,3956.000000,3956.000000,3956.000000,3957.000000,3957.000000,3957.000000,9447.000000,9447.000000,9462.000000,20742.000000,9462.000000,9462.000000,9447.000000,9447.000000,9462.000000,9462.000000,9447.000000,9447.000000,9447.000000,9447.000000,20742.000000,9462.000000,9447.000000,9447.000000,9447.000000,9462.000000,9462.000000,9447.000000,9447.000000,9462.000000,9462.000000,20742.000000,9447.000000,9447.000000,9447.000000,9447.000000,9462.000000,9462.000000,9447.000000,9462.000000,9462.000000,9447.000000,10233.000000,20742.000000,9462.000000,9462.000000,9447.000000,9447.000000,9447.000000,9447.000000,9447.000000,9447.000000,9462.000000,9462.000000,20742.000000,9462.000000,9447.000000,9447.000000,9447.000000,9447.000000,9447.000000,9447.000000,8769.000000,8769.000000,8769.000000,20742.000000,9438.000000,9438.000000,9438.000000,8769.000000,8769.000000,8769.000000,8769.000000,9438.000000,8769.000000,8769.000000,20742.000000,8769.000000,8769.000000,8769.000000,8769.000000,9438.000000,8769.000000,8769.000000,8769.000000,9438.000000,9438.000000,20742.000000,8769.000000,8769.000000,8769.00

In [189]:
df_list = df_list.drop(df_list.index[[0, 1, 2, 3, 15]])

In [190]:
df_list

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,dist1,dist2,D2,D3,D11,D12,D13,D14,D5,D6,D7,D8,D9,V1,V10,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V11,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V12,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V13,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V14,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V15,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V16,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V17,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V18,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V19,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V2,V20,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V21,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V22,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V23,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V24,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V25,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V26,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V27,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V28,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V29,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V3,V30,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V31,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V32,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V33,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,V34,V35,V36,V37,V38,V39,V4,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V5,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V6,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V7,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V8,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V9,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99
10%,1.00,1.00,0.0,0.0,0.0,1.00,0.0,0.0,0.00,0.0,1.0,0.00,0.00,0.0,1.00,0.0,1.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.208333,0.041666,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.00000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,1.00,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.00,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.00,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20%,1.00,1.00,0.0,0.0,0.0,1.00,0.0,0.0,0.00,0.0,1.0,0.00,1.00,1.0,2.00,4.0,4.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.708333,0.125000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.00000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,1.00,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.00,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,

In [162]:
df_list = []
for var in cv_impute_num:
        df = print(np.percentile(X_train[var].dropna(), [10, 20]), max(X_train[var]))
#             df = pd.DataFrame(data_json['query']['results']['quote'])
        df_list.append(df)

[1. 1.] 4668.0
[1. 1.] 5624.0
[0. 0.] 8.0
[0. 0.] 2240.0
[0. 0.] 331.0
[1. 1.] 2240.0
[0. 0.] 2242.0
[0. 0.] 3317.0
[0. 0.] 192.0
[0. 0.] 3244.0
[1. 1.] 3170.0
[0. 0.] 3170.0
[0. 1.] 2903.0
[0. 1.] 1417.0
[1. 2.] 4942.0
[0. 4.] nan
[1. 4.] nan
[0. 0.] 674.0
[0. 0.] 666.0
[0. 0.] nan
[0. 0.] nan
[0. 0.] nan
[0. 0.] nan
[0. 0.] nan
[0. 0.] nan
[0.208333   0.70833302] nan
[0.041666 0.125   ] nan
[1. 1.] 1.0
[0. 0.] 4.0
[0. 0.] 24.0
[0. 0.] 781.0
[0. 0.] 1285.0
[0. 0.] 920.0
[0. 0.] 15.0
[0. 0.] 99.0
[0. 0.] 50.0
[1. 1.] 1.0
[1. 1.] 5.0
[1. 1.] 5.0
[0. 0.] 4.0
[1. 1.] 5.0
[1. 1.] 5.0
[1. 1.] 5.0
[1. 1.] 5.0
[1. 1.] 6.0
[1. 1.] 6.0
[1. 1.] 6.0
[1. 1.] 2.0
[1. 1.] 2.0
[1. 1.] 2.0
[0. 0.] 3.0
[1. 1.] 3.0
[1. 1.] 3.0
[1. 1.] 3.0
[1. 1.] 11.0
[1. 1.] 11.0
[1. 1.] 11.0
[0. 0.] 85573.0
[0. 0.] 146097.0
[0. 0.] 103047.0
[0. 0.] 2594.949951171875
[0. 0.] 5.0
[0. 0.] 11200.0
[0. 0.] 3981.0
[0. 0.] 84408.0
[0. 0.] 133915.0
[0. 0.] 98028.0
[0. 0.] 6000.0
[0. 0.] 10065.0
[0. 0.] 6000.0
[0. 0.] nan
[0. 

In [132]:
# for i in cv_impute_num:
for i in range(1):
    row = num_df.iloc[i]
    next_row = num_df.iloc[i+1]
    if (row == 1):
        print(row)
#         print(next_row)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [97]:
# for i in cv_impute_num:
for j in range(len(num_df)):
#     prev = [i-1] 
    cur = [i] 
    nxt = i+1
    if (num_df.iloc[j, 0] == num_df.iloc[nxt, 0]):
        print(num_df.iloc[j, 0]) 

TypeError: can only concatenate str (not "int") to str

In [75]:
x = 1
print(eval('x + 1'))

2


## <font color='green'> Missing Value Imputaion

In [21]:
from fast_ml.missing_data_imputation import MissingDataImputer_Categorical, MissingDataImputer_Numerical

#### <font color='brown'> Missing Value Imputation - Categorical

**Custom Value Imputer for Numeric Type Categorical Variable : Custom Value - '9999'**

In [22]:
cat_imputer = MissingDataImputer_Categorical(method = 'custom_value', value = '9999', add_indicator= False)
cat_imputer.fit(X_train, variables = cv_impute_num)

**Custom Value Imputer for Character Type Categorical Variable : Custom Value - 'Missing'**

In [23]:
cat_imputer1 = MissingDataImputer_Categorical(method = 'custom_value', value = 'Missing', add_indicator= False)
cat_imputer1.fit(X_train, variables = cv_impute_char)

**Transform Train Data**

In [24]:
X_train = cat_imputer.transform(X_train)

In [25]:
X_train = cat_imputer1.transform(X_train)

#### <font color='brown'> Missing Value Imputation - Numerical

**Missing Imputation for Numerical Variables - "Median"**

In [26]:
num_imputer = MissingDataImputer_Numerical(method = 'median', add_indicator= False)
num_imputer.fit(X_train, num_impute)

**Transform Train Data**

In [27]:
X_train = num_imputer.transform(X_train)

## <font color='green'> **Feature Engineering**

In [28]:
from fast_ml.feature_engineering import FeatureEngineering_Categorical

### <font color='brown'> Manual Encoding - Categorical Variables - Train Data

In [29]:
X_train = mail_transform(X_train, 'P_emaildomain')
X_train = mail_transform(X_train, 'R_emaildomain')
X_train = OS_transform(X_train, 'id_30')
X_train = browser_transform(X_train, 'id_31')
X_train = resol_transform(X_train, 'id_33')
X_train = device_transform(X_train, 'DeviceInfo')

### <font color='brown'> Rare Encoding for categories less than 5%

In [30]:
rare_encoder = FeatureEngineering_Categorical(method='rare')
rare_encoder.fit(X_train, variables = rare_5, rare_tol=5)

In [31]:
X_train = rare_encoder.transform(X_train)

### <font color='brown'> Rare Encoding for categories less than 10%

In [32]:
rare_encoder1 = FeatureEngineering_Categorical(method='rare')
rare_encoder1.fit(X_train, variables = rare_10, rare_tol=10)

In [33]:
X_train = rare_encoder1.transform(X_train)

### <font color='brown'> Categorical Encoding - Method = 'Label'

In [34]:
cat_encoder = FeatureEngineering_Categorical(method = 'label', model = 'classification')
cat_encoder.fit(X_train, variables = cat_var, target = 'isFraud')

**Transform Train & Validation Data**

In [35]:
X_train = cat_encoder.transform(X_train)

### <font color='brown'> Dropping the Variables which are listed

In [37]:
X_train.drop(drop_var, axis = 1)

SyntaxError: invalid syntax (<ipython-input-37-c4c3a09164eb>, line 1)

In [38]:
def transform_data(df):
    df = cat_imputer.transform(df)
    df = cat_imputer1.transform(df)
    df = num_imputer.transform(df)
    df = mail_transform(df, 'P_emaildomain')
    df = mail_transform(df, 'R_emaildomain')
    df = OS_transform(df, 'id_30')
    df = browser_transform(df, 'id_31')
    df = resol_transform(df, 'id_33')
    df = device_transform(df, 'DeviceInfo')
    df = rare_encoder.transform(df)
    df = rare_encoder1.transform(df)
    df = cat_encoder.transform(df)

In [39]:
transform_data(X_val)

## <font color='green'> Model Development

### <font color='brown'> Random Forest

In [41]:
from sklearn.ensemble import RandomForestClassifier

In [42]:
clf = RandomForestClassifier( bootstrap = True,
                              max_depth = 200,
                              max_features = 'auto',
                              max_leaf_nodes = None,
                              min_impurity_decrease = 0.0,
                              min_impurity_split = None,
                              min_samples_leaf = 1,
                              min_samples_split = 2,
                              min_weight_fraction_leaf = 0.0,
                              n_estimators = 10,
                              n_jobs = 1,
                              oob_score = False,
                              random_state = 42,
                              verbose = 0,
                              warm_start = False)
clf.fit(X = X_train,y =  y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=200, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
                       oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 2)]
# Number of features to consider at every split
max_features = ['auto']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(100, 200, num = 2)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2]
# Minimum number of samples required at each leaf node
min_samples_leaf = [4]
# Method of selecting samples for training each tree
bootstrap = [True]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
# rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = clf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X = X_train,y =  y_train)

### <font color='brown'> LightGBM

In [43]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, classification_report, confusion_matrix, roc_curve, auc
import lightgbm as lgb

In [44]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, oob_score=True, random_state=123456)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=True, random_state=123456,
                       verbose=0, warm_start=False)

In [ ]:
from sklearn.metrics import accuracy_score
predicted = rf.predict(X_val)
accuracy = accuracy_score(y_val, predicted)
print(f'Out-of-bag score estimate: {rf.oob_score_:.3}')
print(f'Mean accuracy score: {accuracy:.3}')

### <font color='brown'> XGBoost

In [48]:
from xgboost import XGBClassifier

In [49]:
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)